In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) <100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,14957451.67,32.79,41.35,-34.80,-0.30,-1.41,-0.07
4,0.0,32.79,1010887.07,-16.12,-8.94,4.97,0.59,0.10
5,0.0,41.35,-16.12,418478.55,1.36,2.34,1.47,-0.12
6,0.0,-34.80,-8.94,1.36,106004.06,-0.14,-0.04,-0.04
7,0.0,-0.30,4.97,2.34,-0.14,20604.05,0.11,0.03
8,0.0,-1.41,0.59,1.47,-0.04,0.11,1954.03,0.01
9,0.0,-0.07,0.10,-0.12,-0.04,0.03,0.01,94.97


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_20480/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,0.00001,0.00002,-0.00003,-0.00000,-0.00001,-0.00000
4,NaN,0.00001,1.00000,-0.00002,-0.00003,0.00003,0.00001,0.00001
5,NaN,0.00002,-0.00002,1.00000,0.00001,0.00003,0.00005,-0.00002
6,NaN,-0.00003,-0.00003,0.00001,1.00000,-0.00000,-0.00000,-0.00001
7,NaN,-0.00000,0.00003,0.00003,-0.00000,1.00000,0.00002,0.00002
8,NaN,-0.00001,0.00001,0.00005,-0.00000,0.00002,1.00000,0.00002
9,NaN,-0.00000,0.00001,-0.00002,-0.00001,0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

93.51313213782753

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.79687428e+13 1.25871086e+11 2.07914418e+10 1.28337391e+09
 4.54123979e+07 3.63475861e+05 6.43879360e+02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.0,0.999945,-0.009694,-0.003994,-0.001014,-0.000197,-0.000019,-9.109836e-07
1,-0.0,0.009313,0.996415,-0.082297,-0.016923,-0.003242,-0.000308,-1.483754e-05
2,-0.0,0.004703,0.081168,0.995091,-0.055519,-0.009981,-0.000941,-4.619026e-05
3,0.0,0.001419,0.021162,0.053379,0.997034,-0.051018,-0.004612,-2.245453e-04
4,-0.0,0.000345,0.005108,0.012362,0.050155,0.998133,-0.032135,-1.542005e-03
5,0.0,0.000044,0.000644,0.001553,0.006147,0.031799,0.999176,-2.439648e-02
6,-0.0,0.000003,0.000047,0.000114,0.000448,0.002304,0.024333,9.997011e-01
7,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.9906868190636825,
 0.9188318913486019,
 0.9466211302361076,
 0.9498450087270557,
 0.9682005252264223,
 0.9756669825803272,
 1.0]